In [1]:
import pandas as pd
import math
import re


In [2]:
df=pd.read_csv("../data/CometLandingFixed.csv", 
                dtype={"id_str": str, "in_reply_to_user_id_str": str, "from_user_id_str": str,
                        "in_reply_to_status_id_str": str, "user_followers_count": "Int64",
                        "user_friends_count": "Int64", "geo_coordinates" : str}, parse_dates=['created_at']
              )

In [3]:
df

,id_str,from_user,text,created_at,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
0,540930409279008000,amika0078788556,RT @VersaTechnology: Congratulations @Philae20...,2014-12-05 18:07:14+00:00,NaN,en,NaN,NaN,1297570116,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/3555068293...,425,113,http://twitter.com/amika0078788556/statuses/54...,"{""hashtags"":[{""text"":""Philae"",""indices"":[49,56..."
1,540930042700656000,ChrisDMarshall,CometWatch 2 December Â» Rosetta - ESA's comet...,2014-12-05 18:05:46+00:00,NaN,en,NaN,NaN,42201214,NaN,"<a href=""https://dev.twitter.com/docs/tfw"" rel...",http://pbs.twimg.com/profile_images/228087980/...,347,1286,http://twitter.com/ChrisDMarshall/statuses/540...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
2,540929998388211000,MHuuskoL,RT @EUCouncil: After the #CometLanding - Astro...,2014-12-05 18:05:36+00:00,NaN,en,NaN,NaN,574434762,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://pbs.twimg.com/profile_images/4973510588...,215,310,http://twitter.com/MHuuskoL/statuses/540929998...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
3,540929346878599000,SaraGomezAranci,RT @EUCouncil: After the #CometLanding - Astro...,2014-12-05 18:03:00+00:00,NaN,fr,NaN,NaN,1057356487,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/5391705804...,741,310,http://twitter.com/SaraGomezAranci/statuses/54...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
4,540929229052203000,CBCDay6,RT @shaunmajumder: Feels good to be the @CBCDa...,2014-12-05 18:02:32+00:00,NaN,en,NaN,NaN,177276897,NaN,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/4590694303...,4331,2098,http://twitter.com/CBCDay6/statuses/5409292290...,"{""hashtags"":[{""text"":""MiniMansbridge"",""indices..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77303,532460149084930000,ABForScience,This means that the actual landing will be ar...,2014-11-12 09:09:26+00:00,NaN,en,NaN,NaN,2333122519,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",http://pbs.twimg.com/profile_images/4320695593...,155,212,http://twitter.com/ABForScience/statuses/53246...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77304,532460148451581000,atieyK,RT @ObservingSpace: Weâ€™ve been waiting 10 ye...,2014-11-12 09:09:26+00:00,NaN,en,NaN,NaN,58303715,NaN,"<a href=""https://twitter.com/download/android""...",http://pbs.twimg.com/profile_images/5057311637...,354,1088,http://twitter.com/atieyK/statuses/53246014845...,"{""hashtags"":[{""text"":""cometlanding"",""indices"":..."
77305,532460148254470000,j0nny5,RT @maxplanckpress: Accomazzo (flight director...,2014-11-12 09:09:26+00:00,NaN,en,NaN,NaN,17914863,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/89991673/j...,92,63,http://twitter.com/j0nny5/statuses/53246014825...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77306,532460148238090000,nsentse,7 hours of waiting #CometLanding,2014-11-12 09:09:26+00:00,NaN,en,NaN,NaN,20434005,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/3693655850...,200,267,http://twitter.com/nsentse/statuses/5324601482...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."


In [4]:
len(df)

77308

In [5]:
df.dtypes

id_str                                    object
from_user                                 object
text                                      object
created_at                   datetime64[ns, UTC]
geo_coordinates                           object
user_lang                                 object
in_reply_to_user_id_str                   object
in_reply_to_screen_name                   object
from_user_id_str                          object
in_reply_to_status_id_str                 object
source                                    object
profile_image_url                         object
user_followers_count                       Int64
user_friends_count                         Int64
status_url                                object
entities_str                              object
dtype: object

In [6]:
df[df.duplicated()].head(20)

,id_str,from_user,text,created_at,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str


In [7]:
df = df.drop_duplicates()
df = df.dropna(axis = 0, how = 'all')

In [8]:
reply_df = df[["in_reply_to_user_id_str","in_reply_to_screen_name","in_reply_to_status_id_str"]]

In [9]:
reply_df

,in_reply_to_user_id_str,in_reply_to_screen_name,in_reply_to_status_id_str
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
77303,NaN,NaN,NaN
77304,NaN,NaN,NaN
77305,NaN,NaN,NaN
77306,NaN,NaN,NaN


In [10]:
#reply
def isreply(tweet):
    if not pd.isna(tweet):
#         reply =[]
#         reply.append(tweet)
        print(tweet)
#         return reply
        
#     if tweet["in_reply_to_user_id_str"] == 0:
#         return 0
#     return tweet["in_reply_to_user_id_str"]


In [11]:
r = df[ pd.notna(df["in_reply_to_user_id_str"])] #replies

In [12]:
len(r.index) # amount of replies

1724

In [13]:
# replies
replies = reply_df["in_reply_to_user_id_str"].map(isreply)

11348282
19442947
253536357
253536357
64643056
33602654
1691515789
19402238
529722897
2294593507
41333964
253536357
66162716
59220577
253536357
1273487113
253536357
17842366
26030180
24129206
14526045
21436960
1969346438
253536357
253536357
2232463302
1438925534
15808765
472670056
2590415228
254659730
17384099
177594666
46959037
32520240
443080433
412396499
208442526
86070159
21436960
253536357
253536357
253536357
253536357
208442526
35233873
376790816
404818828
14800111
622113
1266281
2294593507
372434360
329600057
972651
208442526
168090600
246693449
27650674
208442526
1941637627
43024906
86336234
208442526
19903360
2868449663
47306483
371152085
2894572724
106442779
208442526
364423298
253536357
43024906
85751142
208442526
4620451
3108351
41712825
75052666
5402612
2254751592
629637909
21584325
110462645
208442526
5402612
14606395
208442526
75052666
1041442471
119473816
1435461
208442526
229632990
75052666
34800396
32498911
18505048
208442526
208442526
129673790
75052666
114085696
673

In [14]:
text = df["text"]

In [15]:
rt = df[df['text'].apply(lambda x: True if re.search('^RT @.*', x) else False)]
tweet = df[df['text'].apply(lambda x: False if re.search('^RT @.*', x) else True)]

In [16]:
len(rt.index)

59960

In [17]:
len(tweet.index)

17348

In [18]:
twitts_ids = df['id_str'].unique()

In [19]:
len(twitts_ids)

77267

In [20]:
user_ids = df["from_user_id_str"].unique()

In [21]:
len(user_ids)

50217

In [22]:
df[df["id_str"].duplicated()].head(20)

,id_str,from_user,text,created_at,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
387,540156649567436000,msbentley,RT @ESA_Rosetta: Moving back towards a 20 km o...,2014-12-03 14:52:35+00:00,NaN,en,NaN,NaN,10408632,NaN,"<a href=""https://about.twitter.com/products/tw...",http://pbs.twimg.com/profile_images/3698761995...,1387,704,http://twitter.com/msbentley/statuses/54015664...,"{""hashtags"":[{""text"":""67P"",""indices"":[58,62]},..."
6473,532667596664213000,WhatayaNuts,"Hey, glad we're not the only ones wondering! #...",2014-11-12 22:53:46+00:00,NaN,en,NaN,NaN,110812952,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",http://pbs.twimg.com/profile_images/3189840928...,566,1534,http://twitter.com/WhatayaNuts/statuses/532667...,"{""hashtags"":[{""text"":""whatayanuts"",""indices"":[..."
18072,532609776531030000,SuzanneSimnick,RT @Philae2014: Touchdown! Min nya adress: 67P...,2014-11-12 19:04:00+00:00,NaN,en,NaN,NaN,388381588,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/5218234702...,664,1990,http://twitter.com/SuzanneSimnick/statuses/532...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
20545,532595473748860000,Marvellousmimi,RT @BBCBreaking: The first picture from @phila...,2014-11-12 18:07:10+00:00,NaN,en,NaN,NaN,37949599,NaN,"<a href=""https://about.twitter.com/products/tw...",http://pbs.twimg.com/profile_images/3788000006...,2906,131,http://twitter.com/Marvellousmimi/statuses/532...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
22059,532595033317191000,GrillMasterWeb,RT @ESA_Rosetta: Hang on in there Philae! MT @...,2014-11-12 18:05:25+00:00,NaN,en,NaN,NaN,444430537,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/5271607791...,143,132,http://twitter.com/GrillMasterWeb/statuses/532...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
22738,532594796758847000,Nitrobencen0,"RT @esaoperations: Meanwhile, @ESA_Rosetta set...",2014-11-12 18:04:29+00:00,NaN,es,NaN,NaN,428926401,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/4654179300...,1767,620,http://twitter.com/Nitrobencen0/statuses/53259...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
25094,532594014848299000,ScottWx_TWN,RT @ESA_Rosetta: Great shot! MT @Philae2014: ....,2014-11-12 18:01:22+00:00,NaN,en,NaN,NaN,2444371909,NaN,"<a href=""https://about.twitter.com/products/tw...",http://pbs.twimg.com/profile_images/5199398717...,494,744,http://twitter.com/ScottWx_TWN/statuses/532594...,"{""hashtags"":[{""text"":""67P"",""indices"":[90,94]},..."
25583,532580697987629000,benjiderooij,RT @SarcasticRover: Lander did not fire harpoo...,2014-11-12 17:08:27+00:00,NaN,en,NaN,NaN,234522822,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/1681232020...,66,191,http://twitter.com/benjiderooij/statuses/53258...,"{""hashtags"":[{""text"":""cometlanding"",""indices"":..."
25939,532580615770472000,AndCuteness,RT @onl2forme2rand2: RT @ESA_Rosetta: TOUCHDOW...,2014-11-12 17:08:08+00:00,NaN,en,NaN,NaN,2877077292,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/5260783908...,16,38,http://twitter.com/AndCuteness/statuses/532580...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
27424,532580291404365000,Shado_Temple,RT @ESA_Rosetta: Hang on in there Philae! MT @...,2014-11-12 17:06:51+00:00,NaN,en,NaN,NaN,1574456282,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/4869513801...,373,235,http://twitter.com/Shado_Temple/statuses/53258...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."


In [23]:
average_user_tweets = df.groupby(["from_user_id_str"]).agg(["count"])["from_user"]

In [24]:
average_user_tweets.mean()

count    1.539489
dtype: float64

In [25]:
average_user_tweets.sort_values(by="count",ascending=False)

,count
from_user_id_str,
2851551783,116
356021744,99
32427002,88
619629656,82
1401527256,81
...,...
2300275412,1
230038610,1
23006973,1


In [26]:
type(average_user_tweets)

pandas.core.frame.DataFrame

In [27]:
man = average_user_tweets["count"].tolist()

In [28]:
sum(man)/len(man)

1.5394894057670863

In [29]:
user_ids.tolist()[0]

'1297570116'

In [30]:

index = average_user_tweets.index
# print(index)
a_list = list(index)


In [31]:
#user_ids.tolist()


[item for item in user_ids.tolist() if item not in a_list]

[nan]

In [32]:
df.loc[pd.isna(df['from_user_id_str'])] #Nan problem

,id_str,from_user,text,created_at,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
19354,532609094092603000,Gemjacks,RT @NASAJPL: See @Philae2014's view on the way...,2014-11-12 19:01:18+00:00,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN


In [33]:
source = df.groupby(["source"]).agg(["count"])["id_str"]

In [34]:
source

,count
source,
"<a href=""Http://WWW.tweetpooh.cl/download"" rel=""nofollow"">Miguebot</a>",1
"<a href=""http://011.ru"" rel=""nofollow"">severok</a>",6
"<a href=""http://SuperNoder.com/TrendTweeter.php"" rel=""nofollow"">TrendTweeter</a>",1
"<a href=""http://aaplab.com"" rel=""nofollow"">Robird for Android</a>",4
"<a href=""http://actiance.com/support/socialite.aspx"" rel=""nofollow"">Socialite Archiver</a>",1
...,...
"<a href=""https://www.google.fr"" rel=""nofollow"">FirmYn's twit</a>",1
"<a href=""https://www.imanindependentdeveloperwhodontneednowebsite.gov"" rel=""nofollow"">suckmyspiderdickbot</a>",18
"<a href=""https://www.twitter.com/desinewsytweets"" rel=""nofollow"">India News As It Happens</a>",1


In [35]:
source_raw = source.sort_values(by="count",ascending=False)

In [36]:
source_raw

,count
source,
"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",27943
"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",13750
"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",12773
"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",4081
"<a href=""http://twitter.com/#!/download/ipad"" rel=""nofollow"">Twitter for iPad</a>",3283
...,...
"<a href=""http://viralizzer.com"" rel=""nofollow"">ViraliZzer</a>",1
"<a href=""http://twtkr.com"" rel=""nofollow"">twtkr</a>",1
"<a href=""http://twitterstreams.nl"" rel=""nofollow"">rettiwtstreams</a>",1


In [37]:
b = source_raw.index
b_list= list(b)

In [38]:
type(source_raw["count"][1])

numpy.int64

In [39]:
source_raw["count"][1]

13750

In [40]:
b_list

['<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
 '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 '<a href="http://dlvr.it" rel="nofollow">dlvr.it</a>',
 '<a href="https://dev.twitter.com/docs/tfw" rel="nofollow">Twitter for Websites</a>',
 '<a href="http://tapbots.com/tweetbot" rel="nofollow">Tweetbot for iÎŸS</a>',
 '<a href="http://www.twitter.com" rel="nofollow">Twitter for Windows Phone</a>',
 '<a href="http://www.ajaymatharu.com/" rel="nofollow">Tweet Old Post</a>',
 '<a href="http://itunes.apple.com/us/app/twitter/id409789998?mt=12" rel="nofollow">Twitter for Mac</a>',
 '<a href="https://twitter.com/download/android" rel="nofollow">Twitter for Android Tablets</